### `BoundedQueue<T> extends Iterable<T>`

Simply have our `BoundedQueue` method to extends `Iterable<T>`. Make sure to add the Iterator<T> iterator() method so that we can `Override` it in `ArrayRingBuffer`.

In [ ]:
public interface BoundedQueue<T> extends Iterable<T> {
    ...
    Iterator<T> iterator();
}

### `ArrayRingBuffer` Iterator

First, we `@Override` the `iterator()` method that is in the interface. Have this method returns an `ArrayRingBufferIterator()` object.

In [ ]:
@Override
public Iterator<T> iterator() {
    return new ArrayRingBufferIterator();
}

Now create the private class `ArrayRingBufferIterator` that implements `Iterator<T>`

In [ ]:
private class ArrayRingBufferIterator implements Iterator<T>{
    
}

It might be difficult to think about how to implement this class at first, since we can't do the way it was done in the lecture! We have a pointer that cycle back to the first index if it gets to the last index of the array.

The answer: Implement it the same way we implemented `ArrayRingBuffer`
* `first`
* `fillCount`
* `hasNext()` simply checks whether this iterator `fillCount` is 0

In [ ]:
private int firstIterator;
private int fillCountIterator;

public ArrayRingBufferIterator() {
    firstIterator = first;
    fillCountIterator = fillCount;
}

public boolean hasNext() {
    return (fillCountIterator != 0);
}

`next()` method implementation is very similar to `dequeue`.
* Check `hasNext()`. If false, return `NoSuchElementException`
* Otherwise,
    * Assign the element of index `[firstIterator]` to a temporary variable
    * Increment `firstIterator`
    * Decrement `fillCountIterator`
    * If `firstIterator` goes past the last index, reset it to 0
    * and return whatever value was bound to the temporary variable.

In [ ]:
public T next() {
    if (!hasNext()) {
        throw new NoSuchElementException();
    }
    T toBeNext = (T) rb[firstIterator];
    firstIterator++;
    fillCountIterator--;
    if (firstIterator > capacity() - 1) {
        firstIterator = 0;
    }
    return toBeNext;
}


### `equals`

We can't just compare 2 `ArrayRingBuffer` objects because each `ArrayRingBuffer` is an object containing the array `buffer`. On top of that, we can't simply take the `buffer` as it will violate abstractions.

Instead, we can use the iterator to `next()` each element of both `ArrayRingBuffer`. 

1. Assuming that the input object is an `ArrayRingBuffer`, we cast this object as one.
2. For preliminary check, check if both ARB's `fillCount` are the same.
3. If #2 is passed, assign the 2 iterators separately because we'll need to access their elements one-by-one and compare them on the same time.
4. Comparison stage:
    * Occurs as long as any of the iterator still has `hasNext()`
    * Each ARB can contain any kind of object as its element (int, double, string, char, etc.). To be able to compare the elements universally,
        * Convert them to string using `String.valueOf`
        * Then compare using `.equals`.

In [ ]:
@Override
public boolean equals(Object o) {
    // Cast the input object as an ArrayRingBuffer object
    ArrayRingBuffer other = (ArrayRingBuffer) o;
    /**
     * if the fillCount is different, they are not equal
     */
    if (fillCount() != other.fillCount()) {
        return false;
    }
    // Assign the iterators each to separate variables
    Iterator thisIter = this.iterator();
    Iterator otherIter = other.iterator();
    while (thisIter.hasNext()) {
        /**
         * Since the array can contain any type of object, convert it to
         * String for comparison purpose
         */
        if (!String.valueOf(thisIter.next()).equals(String.valueOf(otherIter.next()))) {
            return false;
        }
    }
    return true;
}
